In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install -U langchain-huggingface


In [4]:
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers
!pip install numpy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [5]:
!pip install streamlit pyngrok transformers accelerate bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 107.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 8.2 MB/s eta 0:00:00


In [6]:
!pip install pyngrok

In [ ]:
!ngrok config add-authtoken ****************************

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
%%writefile rag_ui.py

import streamlit as st
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import faiss
import pickle
import numpy as np

# ✅ Toujours au tout début
st.set_page_config(page_title="نظام RAG عربي", layout="centered")

# === Initialiser l'historique ===
if "history" not in st.session_state:
    st.session_state.history = []

# === Choix de modèle (chemins) ===
model_paths = {
    "distilgpt2": "/content/drive/MyDrive/googlesmall",
    "aragpt2-medium": "/content/drive/MyDrive/merged_aragpt2-medium_arabic",
    "gptneo": "/content/drive/MyDrive/final_model_gptneo_arabic_gen",

}

# === Sidebar : sélection de modèle + paramètres ===
with st.sidebar:
    st.header("⚙️ إعدادات")
    selected_model_name = st.selectbox("اختر النموذج:", list(model_paths.keys()))
    top_k = st.slider("عدد الوثائق المسترجعة:", 1, 10, 3)
    alpha = st.slider("نسبة الدمج (FAISS / BM25):", 0.0, 1.0, 0.5, 0.1)
    max_tokens = st.slider("عدد الكلمات في الجواب:", 10, 200, 50)

    st.markdown("---")
    st.header("🗂️ سجل الأسئلة")
    if st.session_state.history:
        for i, (q, r) in enumerate(reversed(st.session_state.history[-5:]), 1):
            st.markdown(f"**{i}. سؤال:** {q}")
            st.markdown(f"👉 {r}")
    else:
        st.write("لا يوجد سجل بعد.")

# === Chargement des ressources ===
@st.cache_resource
def load_resources(model_name):
    model_path = model_paths[model_name]
    faiss_path = "/content/drive/MyDrive/arabic_qa_generation_faiss"

    model = AutoModelForCausalLM.from_pretrained(model_path).to("cpu")
    tokenizer = AutoTokenizer.from_pretrained(model_path)




    embedder = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

    index = faiss.read_index(f"{faiss_path}/faiss.index")
    with open(f"{faiss_path}/documents.pkl", "rb") as f:
        documents = pickle.load(f)

    tokenized_corpus = [doc.split() for doc in documents]
    bm25 = BM25Okapi(tokenized_corpus)

    return tokenizer, model, embedder, index, bm25, documents

tokenizer, model, embedder, index, bm25, documents = load_resources(selected_model_name)

# === Pipeline RAG Hybride ===
def hybrid_rag_pipeline(query, top_k=3, max_new_tokens=50, alpha=0.5):
    query_embedding = embedder.encode([query], convert_to_numpy=True)
    D, I = index.search(query_embedding, k=top_k*2)
    dense_docs = [(documents[i], D[0][idx]) for idx, i in enumerate(I[0])]

    bm25_scores = bm25.get_scores(query.split())
    bm25_top_indices = np.argsort(bm25_scores)[::-1][:top_k*2]
    sparse_docs = [(documents[i], bm25_scores[i]) for i in bm25_top_indices]

    hybrid_scores = {}
    for doc, score in dense_docs:
        hybrid_scores[doc] = alpha * score
    for doc, score in sparse_docs:
        if doc in hybrid_scores:
            hybrid_scores[doc] += (1 - alpha) * score
        else:
            hybrid_scores[doc] = (1 - alpha) * score

    top_contexts = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)[:top_k]
    combined_context = "\n".join([doc for doc, _ in top_contexts])

    prompt = f"""السياق:
{combined_context}

السؤال: {query}
أجب على السؤال فقط، بجملة قصيرة ودقيقة. لا تكرر السؤال أو السياق.
الإجابة:"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
    inputs = {k: v.to('cpu') for k, v in inputs.items()}
    output = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    answer = decoded.split("الإجابة:")[-1].strip().split("\n")[0].strip(" .،")
    return f"الإجابة: {answer}"

# === Entrée utilisateur et affichage résultat ===
st.title("🧠 نظام الأسئلة والأجوبة (RAG) - استرجاع هجين")

user_question = st.text_input("🖊️ أدخل سؤالك بالعربية:", placeholder="مثال: ما هي عاصمة المغرب؟")

if st.button("🔍 احصل على الإجابة"):
    if user_question.strip():
        with st.spinner("🔎 يتم توليد الإجابة..."):
            result = hybrid_rag_pipeline(user_question, top_k=top_k, max_new_tokens=max_tokens, alpha=alpha)
            st.session_state.history.append((user_question, result))
        st.markdown("### 🎯 الإجابة:")
        st.success(result)
    else:
        st.warning("⚠️ الرجاء إدخال سؤال صالح.")


Writing rag_ui.py


In [9]:
!pkill -f ngrok


In [11]:
!pip install rank_bm25

In [ ]:
from pyngrok import ngrok
public_url = ngrok.connect(8501)
print("🚀 Interface Streamlit :", public_url)

!streamlit run rag_ui.py --server.enableCORS false --server.enableXsrfProtection false


🚀 Interface Streamlit : NgrokTunnel: "https://0310-34-125-114-98.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.125.114.98:8501

2025-07-06 18:03:56.546218: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751825036.584660    4526 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751825036.596780    4526 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-06 18:03:56.634181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in